# Interacting with GPIO from MicroBlaze

In [1]:
from pynq.overlays.base import BaseOverlay
import time
import multiprocessing
import os
import socket

base = BaseOverlay("base.bit")

In [2]:
%%microblaze base.PMODB

#include "gpio.h"
#include "pyprintf.h"

//Function to read the value of a selected pin
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

//Function to turn on/off a selected pin
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}

//Function to reset all pins
void reset_gpio(){
    for (unsigned int i = 0; i < 8; i++){
        write_gpio(i, 0);
    }
}

In [ ]:
def run_pynq_server_R():
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.bind(('192.168.2.99', 12345))
    sock.listen()
    conn, addr = sock.accept()
    with conn:
        print("before pynq R")
        while True:
            print("inside pynq R")
            time.sleep(0.0001)
            data = conn.recv(1024)
            if (data.decode() == 'disconnect'):
                print("disconnect pynq R")
                break
            # logic for RGBLED blinking
    print("after pynq R")

def run_pynq_client_R():
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    while True:
        time.sleep(0.0001)
        if base.buttons[0].read() != 0:
            sock.connect(('192.168.2.1', 54321))
            break
    while True:
        time.sleep(0.0001)
        if base.buttons[3].read() != 0:
            sock.sendall(b'disconnect')
            break
    sock.close()

p_server = multiprocessing.Process(target=run_pynq_server_R)
p_server.start()
os.system("taskset -p -c {} {}".format(0, p_server.pid))

p_client = multiprocessing.Process(target=run_pynq_client_R)
p_client.start()
os.system("taskset -p -c {} {}".format(1, p_client.pid))

p_server.join();
p_client.join();